In [2]:
print("hello world")

hello world


In [3]:
a = 1

In [4]:
b=2+a

In [5]:
print(b)

3


Objective: Create a daily new summarizer app
### How it works:
- Runs every day for a given topic
- Takes the last 24 hours and searches for news across the internet
- Summarizes the news and sends it to me

In [1]:
# Import necessary libraries
import requests
import json
from datetime import datetime, timedelta
from serpapi import GoogleSearch
import litellm
import time
import os

In [3]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env

# Get API keys from environment variables
SERPAPI_KEY = os.environ.get("SERP_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Check if keys are available
if not SERPAPI_KEY or not OPENAI_API_KEY:
    print("Warning: API keys not found in environment variables. Please check your .env file.")

# Set the OpenAI API key for litellm
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# Configure litellm
litellm.set_verbose = False

In [4]:
def get_user_topic():
    """Get the news topic from the user via terminal input"""
    topic = input("Enter the news topic you're interested in: ")
    return topic

# Get the topic
topic = get_user_topic()
print(f"Searching for news on: {topic}")

Searching for news on: Trump vs Zelensky


In [5]:
def search_news(topic, api_key, time_period="1d"):
    """Search for news on the given topic using SerpAPI
    
    Args:
        topic (str): The news topic to search for
        api_key (str): SerpAPI key
        time_period (str): Time period for news (default: "1d" for 1 day)
        
    Returns:
        list: List of news results
    """
    print(f"Searching for news on '{topic}' from the last 24 hours...")
    
    # Configure the search parameters
    params = {
        "engine": "google",
        "q": f"{topic} news",
        "tbm": "nws",  # News search
        "tbs": f"qdr:{time_period}",  # Time period (1d = 1 day)
        "num": 10,  # Number of results
        "api_key": api_key
    }
    
    try:
        # Execute the search
        search = GoogleSearch(params)
        results = search.get_dict()
        
        # Check if we have news results
        if "news_results" in results and results["news_results"]:
            print(f"Found {len(results['news_results'])} news articles")
            return results["news_results"]
        else:
            print("No news results found. Try a different topic or time period.")
            return []
    except Exception as e:
        print(f"Error searching for news: {e}")
        return []

# Search for news on the topic
news_results = search_news(topic, SERPAPI_KEY)

Searching for news on 'Trump vs Zelensky' from the last 24 hours...
Found 10 news articles


In [7]:
def fetch_article_content(url):
    """Fetch the content of a news article from its URL
    
    Args:
        url (str): URL of the news article
        
    Returns:
        str: The HTML content of the article
    """
    try:
        # Set a user agent to avoid being blocked
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.text
    except Exception as e:
        print(f"Error fetching article from {url}: {e}")
        return ""

def process_news_results(news_results, max_articles=10):
    """Process the news results and fetch article content
    
    Args:
        news_results (list): List of news results from SerpAPI
        max_articles (int): Maximum number of articles to process
        
    Returns:
        list: List of processed articles with content
    """
    processed_articles = []
    
    # Limit the number of articles to process
    articles_to_process = news_results[:max_articles]
    
    print(f"Fetching content for {len(articles_to_process)} articles...")
    
    for i, article in enumerate(articles_to_process):
        print(f"Processing article {i+1}/{len(articles_to_process)}: {article['title']}")
        
        # Extract article information
        article_info = {
            "title": article.get("title", ""),
            "link": article.get("link", ""),
            "source": article.get("source", ""),
            "snippet": article.get("snippet", ""),
            "date": article.get("date", "")
        }
        
        # Fetch the article content if we have a link
        if article_info["link"]:
            article_info["content"] = fetch_article_content(article_info["link"])
        else:
            article_info["content"] = ""
        
        processed_articles.append(article_info)
        
        # Add a small delay to avoid overwhelming servers
        time.sleep(1)
    
    return processed_articles

# Process the news results and fetch article content
processed_articles = process_news_results(news_results)

Fetching content for 10 articles...
Processing article 1/10: Zelensky told to leave White House after angry spat with Trump and Vance
Processing article 2/10: Bowen: Trump-Zelensky row signals looming crisis between Europe and US
Processing article 3/10: Zelensky bristled at his berating by Trump and Vance. After three years of war, what did they expect?
Processing article 4/10: ‘The free world needs a new leader’, EU foreign chief warns after explosive row between Trump and Zelensky
Processing article 5/10: In Showdown With Zelensky, Trump Takes Offense on Putin’s Behalf
Processing article 6/10: Contentious Trump-Zelensky meeting threatens U.S. support for Ukraine
Error fetching article from https://www.washingtonpost.com/politics/2025/02/28/trump-ukraine-russia-zelensky/: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=10)
Processing article 7/10: 5 takeaways from Trump-Vance-Zelensky Oval Office blowup
Error fetching article from https://t

In [8]:
def summarize_with_llm(articles, topic):
    """Summarize the news articles using an LLM
    
    Args:
        articles (list): List of processed articles
        topic (str): The original search topic
        
    Returns:
        str: Summarized news in the requested format
    """
    if not articles:
        return "No articles found to summarize."
    
    print("Summarizing news articles using LLM...")
    
    # Prepare the context for the LLM
    context = f"I need a summary of recent news about '{topic}'. Here are the articles I found:\n\n"
    
    for i, article in enumerate(articles):
        context += f"Article {i+1}: {article['title']}\n"
        context += f"Source: {article['source']}\n"
        context += f"Date: {article['date']}\n"
        context += f"Snippet: {article['snippet']}\n\n"
    
    # Create the prompt for the LLM
    prompt = f"""{context}
    
    Based on these articles, provide me with the "Top 3 important items I should know about {topic} and why they matter".
    
    Format your response as a numbered list with a brief explanation for each item.
    Focus on the most significant developments or insights from the last 24 hours.
    """
    
    try:
        # Call the LLM using litellm
        response = litellm.completion(
            model="gpt-4o",  # You can change this to your preferred model
            messages=[
                {"role": "system", "content": "You are a helpful news summarization assistant that provides concise, accurate summaries of recent news."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000
        )
        
        # Extract the summary from the response
        summary = response.choices[0].message.content
        return summary
    except Exception as e:
        print(f"Error summarizing with LLM: {e}")
        return "Error generating summary. Please check your API keys and try again."

# Summarize the articles
summary = summarize_with_llm(processed_articles, topic)

Summarizing news articles using LLM...


In [9]:
def display_summary(summary, topic):
    """Display the final news summary
    
    Args:
        summary (str): The summarized news
        topic (str): The original search topic
    """
    print("\n" + "=" * 80)
    print(f"NEWS SUMMARY FOR: {topic.upper()}")
    print("=" * 80)
    print(summary)
    print("=" * 80)

# Display the summary
display_summary(summary, topic)


NEWS SUMMARY FOR: TRUMP VS ZELENSKY
Certainly! Here are the top three important items to know about the recent Trump vs. Zelensky situation, based on the articles provided:

1. **Heated Exchange in the Oval Office:**
   A meeting between former U.S. President Donald Trump, Ukrainian President Volodymyr Zelensky, and Vice President JD Vance descended into a contentious argument. The confrontation included accusations from Trump that Zelensky was unprepared for peace, reflecting deep tensions about the handling of the ongoing Ukraine-Russia conflict. This matters because it underscores the strained diplomatic relations and potential challenges in obtaining U.S. support for Ukraine.

2. **Implications for U.S.-Ukraine Relations:**
   The fallout from this meeting has raised concerns about the future of U.S. support for Ukraine. Zelensky's visit to Washington was intended to secure continued American backing amid Russia's ongoing military aggression. The hostile interaction signals possib

In [12]:
def news_summarizer(topic=None):
    """Complete news summarizer function that runs the entire process
    
    Args:
        topic (str, optional): The news topic. If None, will prompt the user.
    
    Returns:
        str: The summarized news
    """
    # 1. Get the topic if not provided
    topic = get_user_topic()
    
    print(f"Searching for news on: {topic}")
    
    # 2. Search for news
    news_results = search_news(topic, SERPAPI_KEY)
    
    if not news_results:
        return "No news found for the given topic. Please try a different topic."
    
    # 3. Process the news results and fetch content
    processed_articles = process_news_results(news_results)
    
    # 4. Summarize the articles
    summary = summarize_with_llm(processed_articles, topic)
    
    # 5. Display the summary
    display_summary(summary, topic)
    
    return summary

# Example usage:
summary = news_summarizer("artificial intelligence")

Searching for news on: India EU meet
Searching for news on 'India EU meet' from the last 24 hours...
Found 10 news articles
Fetching content for 10 articles...
Processing article 1/10: India and EU agree to conclude a long-pending free trade agreement by the end of this year
Processing article 2/10: India-EU Trade Council Meeting Boosts Cooperation
Processing article 3/10: India-EU Trade Pact Talks Heat Up Amid Trump Tariff Warnings PM Meets EU Chief In Delhi
Processing article 4/10: India, EU hold bilateral cluster meeting, discuss connectivity, IMEC, security
Processing article 5/10: Joint Statement : Second Meeting of the India-EU TTC
Processing article 6/10: India and EU hold their Trade and Technology Council meeting, stress strengthening cooperation on trade and emerging technologies
Processing article 7/10: Spotlight on FTA ahead of EU chief’s key meetings in India
Processing article 8/10: India a trusted friend, looking to elevate strategic ties: EU President
Processing article